### **Image-Scrapper**

In [ ]:
import os
import time
import requests
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


class EbayImageScraper:
    def __init__(self, url, timeout=7):
        self.url = url
        self.driver = self._initialize_driver()
        self.wait = WebDriverWait(self.driver, timeout=timeout)

    def _initialize_driver(self):
        chrome_options = Options()
        chrome_options.add_argument("--disable-http2")
        chrome_options.add_argument("--incognito")
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_argument("--ignore-certificate-errors")
        chrome_options.add_argument("--enable-features=NetworkServiceInProcess")
        chrome_options.add_argument("--disable-features=NetworkService")
        chrome_options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"
        )
        driver = webdriver.Chrome(options=chrome_options)
        driver.maximize_window()
        return driver

    def _wait_for_page_to_load(self):
        try:
            self.wait.until(
                lambda d: d.execute_script("return document.readyState") == "complete"
            )
        except:
            print("⚠️ Page may not have loaded completely.")
        else:
            print("✅ Page loaded.")

    def access_website(self):
        self.driver.get(self.url)
        self._wait_for_page_to_load()

    def search_products(self, text):
        try:
            search_bar = self.wait.until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="gh-ac"]'))
            )
            search_bar.send_keys(text)
            time.sleep(2)
            search_button = self.wait.until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="gh-search-btn"]'))
            )
            search_button.click()
            self._wait_for_page_to_load()
        except Exception as e:
            print(f"❌ Error during search: {e}")

    def _extract_image_url(self, element):
        try:
            return element.get_attribute("src")
        except:
            return np.nan

    def scrape_images(self, folder_name):
        print(f"\n📸 Scraping images into folder: {folder_name}\n")

        try:
            image_elements = self.wait.until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "img.s-card__image"))
            )
        except:
            print("❌ Couldn't find images on the page.")
            return

        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        for idx, img in enumerate(image_elements):
            img_url = self._extract_image_url(img)
            if img_url and "http" in img_url:
                try:
                    img_data = requests.get(img_url, timeout=10).content
                    filename = os.path.join(folder_name, f"image_{idx + 1}.jpg")
                    with open(filename, 'wb') as f:
                        f.write(img_data)
                    print(f"[✓] Saved: {filename}")
                except Exception as e:
                    print(f"[x] Failed to save image {idx + 1}: {e}")

    def run(self, text="laptop", folder_name="images"):
        try:
            self.access_website()
            self.search_products(text)
            self.scrape_images(folder_name)
        finally:
            time.sleep(2)
            self.driver.quit()
            print("\n🚀 Done scraping and closed the browser.\n")


# ------------------------------------------------------------------- #

if __name__ == "__main__":
    scraper = EbayImageScraper(url="https://www.ebay.com/")
    scraper.run(text="gaming laptop", folder_name="gaming_laptops")
